Análises

1. Pessoas que repetiram a mesma matéria durante o período
2. Seguir o "quadrimestre ideal" do BC&T
3. Matérias escolhidas por pessoas que entraram no mesmo ano
4. Matérias com alto índice de reprovação devem ter maior mistura de alunos de anos de ingresso diferentes.  

Nota: Pros RAs de 8 digitos, o ano de ingresso são os dois ultimos dígitos

Estrutura do código: 
- Blocos individuais para cada "Parte" da execução
- Blocos para a filtragem dos CSVs
- Blocos para a construção dos grafos
- Blocos para plotar os grafos

# O código de verdade começa aqui em baixo


In [151]:
# Inclusão das bibliotecas
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import igraph as ig
import os 

In [152]:
# Coisas aleatorias que podem ser uteis

# lista com os nomes dos arquivos de texto e csv sem o formato. tomara que dê pra iterar por 
# essa lista pra montar os csvs
filenames = ["2022_3_A", "2022_3_B", "2022_3_R", "2023_1_A", "2023_1_R" 
             , "2023_2_A",  "2023_2_R", "2023_3_A", "2023_3_R"]
# "2023_2_B", -> está sem os bixos de 23 pq o pdf é torto

RAs = [] # armazena os RAs dos alunos
Disciplinas = [] # armazena as disciplinas

RAs_por_quad:dict
disciplinass_por_quad:dict

In [160]:
# Código que forma os CSVs mesmo (tirando o dos bixos de 2023 eu não sei oq deu na prograd pra mudar)

for k in filenames:
    auxc = "CSVs/" + k + ".csv"
    auxt = "TXTs/" + k + ".txt"

    t = open(auxc, "w")
    t.write("")
    t.close()

    a = open(auxt, "r", encoding="utf-8")
    b = open(auxc, "a", encoding="utf-8")

    cont = 0 # ponteiro pra saber o a linha mesmo. nn sei pq tem isso

    auxiliarR = []
    auxiliarD = []

    for i in a:

        ins = ""

        cont+= 1

        l = 0 # contador do caracter atual da linha

        if cont == 1: continue # Caso esteja na primeira linha, só pula

        if cont == 2: # Caso esteja na segunda linha, adiciona os indices do CSV
            cab = ""

            while i[l] != 'T': # adiciona todas as letras até o 'T' de "Turma"
                
                if i[l] == " " and l < 6: cab+= "," # se o atual é um espaço, adiciona uma virgula antes

                cab+= i[l]

                l+=1
            
            cab+= "\n"
            b.write(cab)

            continue
        else:
            if not i[0].isnumeric() or i[2] == "/" or i[3] == "/" or i[4] == "/": continue 
            # ^se o primeiro digito for um numero ou for um fim da pagina do pdf ele só pula a escrita

        RA = ""
        disc = ""

        while i[l] != " ": # pega o RA até o primeiro espaço
            RA+= i[l]
            l+=1
        l+=1

        while i[l] != " ": # pega o codigo da turma até o segundo espaço
            disc+= i[l]
            l+=1
        l+=1
        
        # no código da turma, pode valer a pena ver de armazenar o codigo da disciplina só, pq aí
        # conseguimos procurar o nome mesmo no catálogo pra printar bonitinho depois

        if RA not in RAs: RAs.append(RA)
        if RA not in auxiliarR:auxiliarR.append(RA)

        if disc not in Disciplinas: Disciplinas.append(disc)
        if disc not in auxiliarD: auxiliarD.append(disc)

        ins+= RA
        ins+= ", "
        ins+= disc
        ins+= "\n"

        b.write(ins)

    RAs_por_quad[k] = auxiliarR
    disciplinass_por_quad[k] = auxiliarD

    a.close()
    b.close()

In [161]:
grafos:dict

for k in filenames:
    auxc = "CSVs/" + k + ".csv"    

    a = pd.read_csv(auxc) # Lê o csv do grafo

    grafos[k] = nx.Graph()

    for i in range((int)(a.size / 2)):
        grafos[k].add_node((str)(a.loc[i][0]), bipartite = 0)
        grafos[k].add_node((str)(a.loc[i][1]), bipartite = 1)
        grafos[k].add_edge((str)(a.loc[i][0]), (str)(a.loc[i][1]))

    print("{} : {}".format(k, grafos[k]))


2022_3_A : Graph with 11050 nodes and 35071 edges
2022_3_B : Graph with 1947 nodes and 1943 edges
2022_3_R : Graph with 2384 nodes and 2507 edges
2023_1_A : Graph with 12532 nodes and 42532 edges
2023_1_R : Graph with 2253 nodes and 2099 edges
2023_2_A : Graph with 11395 nodes and 36227 edges
2023_2_R : Graph with 2453 nodes and 2463 edges
2023_3_A : Graph with 12975 nodes and 44563 edges
2023_3_R : Graph with 2508 nodes and 2487 edges


In [162]:
# monta um dicionario com o grau de cada vertice individual

a = nx.degree_centrality(grafos[filenames[0]])

a.update((x, y*11049) for x, y in a.items()) # multiplica a centralidade de grau por n-1

print(a)

{'11002112': 2.0, ' NA1BHO1335-15SB': 87.0, ' DB1BHP0202-19SB': 4.0, '11202130002': 4.0, ' DA1BCM0506-15SB': 70.0, ' DA1BCJ0203-15SB': 29.0, ' DA1MCTC002-15SB': 60.0, ' NB1BCN0405-15SB': 40.0, '11202112038': 3.0, ' DB5BIS0005-15SB': 13.0, ' NC1BIL0304-15SB': 36.0, ' NB1NHZ5023-18SA': 11.0, '11201921240': 5.0, ' DC2BIJ0207-15SA': 6.0, ' DA1BCK0103-15SA': 62.0, ' DA1ESZI045-17SA': 30.0, ' DA1BCN0405-15SA': 51.0, ' DA1ESTO012-17SA': 90.0, '21077113': 3.0, ' NA1BCJ0203-15SA': 33.0, ' NA1MCZB015-13SA': 70.0, ' NA1MCTB022-17SA': 45.0, '11202021033': 1.0, ' NA3BCJ0203-15SA': 33.0, '11202130004': 4.0, ' DA1BIR0004-15SA': 86.0, ' DA1BCM0506-15SA': 90.0, ' DA1BIN0406-15SA': 43.0, ' DA1BCL0307-15SA': 30.0, '11202111469': 5.0, ' DA1BIJ0207-15SB': 9.0, ' DA1BIR0004-15SB': 73.0, ' DA1BHO0102-15SB': 90.0, ' DA1BHO1102-19SB': 89.0, ' DA1BIN0406-15SB': 90.0, '11201721638': 4.0, ' NA1MCTB002-13SA': 40.0, ' NA1MCTB007-17SA': 30.0, ' NA2MCTA017-17SA': 72.0, '11049814': 2.0, ' NA1ESZA018-17SA': 20.0, ' NA1

In [163]:
for l in filenames:
    print(len(RAs_por_quad[l]))

10011
1943
1690
11558
1556
10452
1775
11998
1796


In [171]:
# isso projeta o grafo original com os pesos sendo as turmas compartilhadas pelas pessoas

g2 = nx.bipartite.weighted_projected_graph(grafos[filenames[0]], RAs_por_quad[filenames[0]]) # falta aqui a lista de vertices pra dobrar

nx.get_edge_attributes(g2, "weight")

{('11002112', '11202021306'): 1,
 ('11002112', '11201920340'): 1,
 ('11002112', '11202210099'): 1,
 ('11002112', '11201920273'): 1,
 ('11002112', '21093515'): 1,
 ('11002112', '11089911'): 1,
 ('11002112', '11201920624'): 1,
 ('11002112', '11201811774'): 1,
 ('11002112', '11201720742'): 1,
 ('11002112', '11202131648'): 1,
 ('11002112', '11201922018'): 1,
 ('11002112', '11201812217'): 1,
 ('11002112', '11201810924'): 1,
 ('11002112', '11201721741'): 1,
 ('11002112', '11202022025'): 1,
 ('11002112', '11202232278'): 1,
 ('11002112', '11201920436'): 1,
 ('11002112', '11202020249'): 1,
 ('11002112', '11201921286'): 1,
 ('11002112', '11202232386'): 1,
 ('11002112', '21033511'): 1,
 ('11002112', '11202111632'): 1,
 ('11002112', '11201720198'): 1,
 ('11002112', '11202111629'): 1,
 ('11002112', '21077916'): 1,
 ('11002112', '11202020483'): 1,
 ('11002112', '11202210110'): 1,
 ('11002112', '11202210105'): 1,
 ('11002112', '11202020676'): 1,
 ('11002112', '11202021008'): 1,
 ('11002112', '2105111

# abaixo disso são os meus testes iniciais

In [ ]:
# Monta o grafo com o CSV de teste
a = pd.read_csv("test.csv") # Lê o csv do grafo

g = nx.Graph() # Cria o objeto grafo

# contando os RAs unicos
#cout = []
#c = 0

#for i in range((int)(a.size / 2)):
#    if a.loc[i][0] not in cout:
#        c+=1
#        cout+= [a.loc[i][0]]

#print(c)

# Cria os vertices dos RAs no grafo
for i in range((int)(a.size / 2)):
    g.add_node(a.loc[i][0], bipartite = 0)
    g.add_node(a.loc[i][1], bipartite = 1)
    g.add_edge(a.loc[i][0], a.loc[i][1])

# nx.draw_kamada_kawai(g, arrows=False, with_labels=False, node_size=2)

print(g)

# código com o igraph

#G = ig.Graph.from_networkx(g)
#fig, ax = plt.subplots()
#ig.plot(G, target=ax, vertex_size=2, layout = G.layout("kk"))
#plt.show()
#print(g)

In [ ]:
# Código que forma o CSV de teste
t = open("test.csv", "w")
t.write("")
t.close()

a = open("test.txt", "r", encoding="utf-8")
b = open("test.csv", "a", encoding="utf-8")

cont = 0 # ponteiro pra saber o a linha mesmo. nn sei pq tem isso

for i in a:

    ins = ""

    cont+= 1

    l = 0 # contador do caracter atual da linha

    if cont == 1: continue # Caso esteja na primeira linha, só pula

    if cont == 2: # Caso esteja na segunda linha, adiciona os indices do CSV
        cab = ""

        while i[l] != 'T': # adiciona todas as letras até o 'T' de "Turma"
            
            if i[l] == " " and l < 6: cab+= "," # se o atual é um espaço, adiciona uma virgula antes

            cab+= i[l]

            l+=1
        
        cab+= "\n"
        b.write(cab)

        continue
    else:
        if not i[0].isnumeric() or i[2] == "/" or i[3] == "/" or i[4] == "/": continue 
        # ^se o primeiro digito for um numero ou for um fim da pagina do pdf ele só pula a escrita

    RA = ""
    disc = ""

    while i[l] != " ": # pega o RA até o primeiro espaço
        RA+= i[l]
        l+=1
    l+=1

    while i[l] != " ": # pega o codigo da turma até o segundo espaço
        disc+= i[l]
        l+=1
    l+=1
    
    # no código da turma, pode valer a pena ver de armazenar o codigo da disciplina só, pq aí
    # conseguimos procurar o nome mesmo no catálogo pra printar bonitinho depois

    ins+= RA
    ins+= ", "
    ins+= disc
    ins+= "\n"

    b.write(ins)

a.close()
b.close()